# Subtext Codec Demo

In [1]:
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


In [2]:
from subtext_codec import cli

In [3]:
import subtext_codec

In [4]:
from huggingface_hub import notebook_login

In [5]:
notebook_login()

In [6]:
subtext_codec.set_deterministic(0)

In [8]:
tokenizer, model = subtext_codec.load_model_and_tokenizer(
    "meta-llama/Llama-3.1-8B", "cuda"
)


ValueError: Could not find LlamaForCausalLM neither in <module 'transformers.models.llama' from 'C:\\Users\\shevi\\src\\github.com\\shevisj\\subtext-codec\\.venv\\Lib\\site-packages\\transformers\\models\\llama\\__init__.py'> nor in <module 'transformers' from 'C:\\Users\\shevi\\src\\github.com\\shevisj\\subtext-codec\\.venv\\Lib\\site-packages\\transformers\\__init__.py'>!

In [ ]:
cfg = subtext_codec.CodecConfig(
    model_name_or_path="meta-llama/Llama-3.1-8B-Instruct",
    device="cuda",
    base=3,
    prompt_prefix="In response to a video about an adorable cat playing with a leaf: ",
    max_context_length=None,
    top_k=8,
    store_model_in_key=True,
    torch_dtype="bf16",
)

In [31]:
payload = b"hello world my name is shevis"
encoded, key = subtext_codec.encode_data_to_text(payload, cfg, model, tokenizer)
assert key.model_name_or_path == cfg.model_name_or_path

ValueError: Reached max_new_tokens=128 before consuming all digits (128/146)

In [ ]:
print(encoded)

In [ ]:
decoded = subtext_codec.decode_text_to_data(
    encoded,
    key=key,
    prompt_prefix=cfg.prompt_prefix,
    model=model,
    tokenizer=tokenizer,
    device="cuda",
    max_context_length=cfg.max_context_length,
)
assert decoded == payload

'Once upon a time, 20th Century Fox had the idea of creating an all-fantasy, live-action movie franchise, which would have included a movie called “Rapaz”, about the legendary King Kong, “Tiger King,” a movie adaptation'

In [ ]:
noisy_encoded = encoded + " Trailing unrelated text after sentinel."
decoded_with_noise = subtext_codec.decode_text_to_data(
    noisy_encoded,
    key=key,
    prompt_prefix=cfg.prompt_prefix,
    model=model,
    tokenizer=tokenizer,
    device="cpu",
    max_context_length=cfg.max_context_length,
)
assert decoded_with_noise == payload

In [ ]:
trimmed_ids = tokenizer(encoded, return_tensors="pt").input_ids[0][:-1]
trimmed_text = tokenizer.decode(trimmed_ids, skip_special_tokens=True)
subtext_codec.decode_text_to_data(
    trimmed_text,
    key=key,
    prompt_prefix=cfg.prompt_prefix,
    model=model,
    tokenizer=tokenizer,
    device="cuda",
    max_context_length=cfg.max_context_length,
)